In [1]:
from raphtory import Graph
import pandas as pd
import os

# Load data
df_kpi = pd.read_csv("./kiali_kpi_metrics.csv")

# Round timestamp to the nearest minute
df_kpi['timestamp'] = pd.to_datetime(df_kpi['timestamp']).dt.floor('min')

# Convert timestamp to Unix time (seconds since epoch)
df_kpi['timestamp'] = pd.to_datetime(df_kpi['timestamp']).astype(int) // 10**9

# Multi-class labeling
def classify_error(er):
    if er < 0.01:
        return 0
    elif er < 0.15:
        return 1
    else:
        return 2

df_kpi["status"] = df_kpi["error_rate"].apply(classify_error)

# Function to assign color based on status
def get_color(status):
    if status == 0:
        return "green"
    elif status == 1:
        return "orange"
    elif status == 2:
        return "red"

# Create communication graph
g = Graph()
for _, row in df_kpi.iterrows():
    ts = int(row["timestamp"])
    src = str(row["source_workload"])
    dst = str(row["destination_workload"])
    status = row["status"]
    color = get_color(status)  # Get color based on status
    
    # Add nodes
    g.add_node(timestamp=ts, id=src, properties={"role": "source"})
    g.add_node(timestamp=ts, id=dst, properties={"role": "destination"})
    
    # Add edge with color and name based on status
    g.add_edge(
        timestamp=ts,
        src=src,
        dst=dst,
        properties={
            "status": status,
            "throughput": row.get("throughput", 0.0),
            "color": color,  # Add color property
            "name": f"status_{status}"  # Add name property based on status
        }
    )
    print(f"Adding edge: {src} -> {dst}, Properties: {{'status': {status}, 'throughput': {row.get('throughput', 0.0)}, 'color': {color}, 'name': 'status_{status}'}}")
# Save graph
output_dir = "graphs/communication_graph"
if os.path.exists(output_dir):
    import shutil
    shutil.rmtree(output_dir)  # Remove the directory if it already exists
os.makedirs("graphs/", exist_ok=True)
g.save_to_file("graphs/communication_graph")
print("✅ Communication graph saved.")

Adding edge: checkoutservice -> cartservice, Properties: {'status': 2, 'throughput': nan, 'color': red, 'name': 'status_2'}
Adding edge: checkoutservice -> cartservice, Properties: {'status': 0, 'throughput': nan, 'color': green, 'name': 'status_0'}
Adding edge: checkoutservice -> cartservice, Properties: {'status': 2, 'throughput': nan, 'color': red, 'name': 'status_2'}
Adding edge: checkoutservice -> cartservice, Properties: {'status': 2, 'throughput': nan, 'color': red, 'name': 'status_2'}
Adding edge: checkoutservice -> cartservice, Properties: {'status': 2, 'throughput': nan, 'color': red, 'name': 'status_2'}
Adding edge: checkoutservice -> cartservice, Properties: {'status': 0, 'throughput': 539.0, 'color': green, 'name': 'status_0'}
Adding edge: checkoutservice -> cartservice, Properties: {'status': 2, 'throughput': nan, 'color': red, 'name': 'status_2'}
Adding edge: checkoutservice -> cartservice, Properties: {'status': 2, 'throughput': nan, 'color': red, 'name': 'status_2'}
Ad

In [2]:
from raphtory.graphql import GraphServer

# Iniciar el servidor GraphQL
server = GraphServer(work_dir="graphs/").start()
client = server.get_client()

# Función para ejecutar consultas de forma segura
def safe_query(query, label):
    try:
        result = client.query(query)
        if "graph" in result or "__schema" in result:
            print(f"✅ {label}:")
            return result
        else:
            print(f"⚠️ Advertencia en {label}: Datos devueltos pero no en el formato esperado.")
            return result  # Devolver el resultado para inspección
    except Exception as e:
        print(f"❌ Excepción en {label}: {e}")
        return None

# Query 1: Propiedades de los nodos
node_properties_query = """
{
  graph(path: "communication_graph") {
    nodes {
      list {
        name
        degree
        properties {
          keys
          values {
            value
          }
        }
      }
    }
  }
}
"""
node_data = safe_query(node_properties_query, "Node Properties")
if node_data and "graph" in node_data:
    nodes = node_data["graph"]["nodes"]["list"]
    for node in nodes:
        properties = dict(zip(node["properties"]["keys"], [v["value"] for v in node["properties"]["values"]]))
        print(f"Node: {node['name']}, Degree: {node['degree']}, Properties: {properties}")

# Query 2: Propiedades de las aristas
# Query para obtener las propiedades de las aristas
edge_properties_query = """
{
  graph(path: "communication_graph") {
    edges {
      list {
        src {
          name
        }
        dst {
          name
        }
        properties {
          keys
          values {
            value
          }
        }
      }
    }
  }
}
"""

# Ejecutar la query
edge_data = safe_query(edge_properties_query, "Edge Properties")

# Procesar y mostrar los resultados
if edge_data and "graph" in edge_data:
    edges = edge_data["graph"]["edges"]["list"]
    for edge in edges:
        src = edge["src"]["name"]
        dst = edge["dst"]["name"]
        properties = dict(zip(edge["properties"]["keys"], [v["value"] for v in edge["properties"]["values"]]))
        print(f"Edge: {src} -> {dst}, Properties: {properties}")
else:
    print("❌ No se pudieron obtener los datos de las aristas.")

# # Query 3: Nodos con alto grado
# high_degree_nodes_query = """
# {
#   graph(path: "communication_graph") {
#     nodes {
#       list {
#         name
#         degree
#       }
#     }
#   }
# }
# """
# node_degree_data = safe_query(high_degree_nodes_query, "Node Degrees")
# if node_degree_data and "graph" in node_degree_data:
#     high_degree_nodes = [
#         node for node in node_degree_data["graph"]["nodes"]["list"]
#         if node["degree"] > 5
#     ]
#     print("🎯 High Degree Nodes:")
#     for node in high_degree_nodes:
#         print(f"Node: {node['name']}, Degree: {node['degree']}")
test_query = """
{
  graph(path: "communication_graph") {
    edges {
      list {
        src {
          name
        }
        dst {
          name
        }
      }
    }
  }
}
"""
test_data = safe_query(test_query, "Test Query")
print(test_data)

# Query 4: Introspección del esquema
introspection_query = """
{
  __schema {
    types {
      name
      fields {
        name
        type {
          name
          kind
        }
      }
    }
  }
}
"""
schema_info = safe_query(introspection_query, "Schema Introspection")
if schema_info and "__schema" in schema_info:
    types = schema_info["__schema"]["types"]
    for type_info in types:
        print(f"Type: {type_info['name']}")
        if type_info["fields"]:
            for field in type_info["fields"]:
                print(f"  Field: {field['name']}, Type: {field['type']['name']}, Kind: {field['type']['kind']}")

✅ Node Properties:
Node: checkoutservice, Degree: 7, Properties: {'role': 'destination'}
Node: cartservice, Degree: 2, Properties: {'role': 'destination'}
Node: currencyservice, Degree: 2, Properties: {'role': 'destination'}
Node: emailservice, Degree: 1, Properties: {'role': 'destination'}
Node: paymentservice, Degree: 1, Properties: {'role': 'destination'}
Node: productcatalogservice, Degree: 3, Properties: {'role': 'destination'}
Node: shippingservice, Degree: 2, Properties: {'role': 'destination'}
Node: frontend, Degree: 8, Properties: {'role': 'destination'}
Node: adservice, Degree: 1, Properties: {'role': 'destination'}
Node: recommendationservice, Degree: 2, Properties: {'role': 'source'}
Node: loadgenerator, Degree: 1, Properties: {'role': 'source'}
❌ Excepción en Edge Properties: error sending request for url (http://localhost:1736/)
Caused by:
  -> client error (SendRequest)
  -> connection closed before message completed
❌ No se pudieron obtener los datos de las aristas.
✅ T